In [4]:
# Imports
import pandas as pd
import numpy as np
import tifffile as tif
import matplotlib.pyplot as plt
import os, gc, glob
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import cv2
import joblib


## Import the images and preprocess
    #Load Data
    #Reshape image to min X,Y dimensions
    #Resize the images - reduce the size

In [5]:
#Load image from directory
dirname = '/Users/stephanereis/Downloads/temp'  # absolute path
image_list = []   # list which will contain array of individual tif files
# iterate through the file directory
for fname in os.listdir(dirname):
    print(fname)
    im = Image.open(os.path.join(dirname, fname))
    imarray = np.array(im)
    image_list.append(imarray)

6140_1_2.tif
6040_2_2.tif
6140_3_1.tif
6040_4_4.tif
6040_1_0.tif
6040_1_3.tif
6090_2_0.tif
6100_2_3.tif
6100_2_2.tif
6070_2_3.tif
6110_1_2.tif
6100_1_3.tif
6170_2_4.tif
6010_4_4.tif
6170_0_4.tif
6170_4_1.tif
6010_4_2.tif
6160_2_1.tif


In [6]:
# explore the shape of the files
array_shape = []
for i in image_list:
    print(i.shape)
    array_shape.append(i.shape)

(3348, 3396, 3)
(3349, 3391, 3)
(3348, 3396, 3)
(3346, 3387, 3)
(3349, 3391, 3)
(3349, 3391, 3)
(3349, 3393, 3)
(3349, 3391, 3)
(3349, 3391, 3)
(3350, 3338, 3)
(3348, 3396, 3)
(3349, 3391, 3)
(3349, 3389, 3)
(3345, 3393, 3)
(3349, 3389, 3)
(3345, 3393, 3)
(3345, 3396, 3)
(3349, 3393, 3)


In [7]:
# get the min and max values of the shape of the images 
df_shape = pd.DataFrame(array_shape)
df_shape.head(3)

,0,1,2
0,3348,3396,3
1,3349,3391,3
2,3348,3396,3


In [8]:
# X shape min
x_min = df_shape[0].min()
x_min

3345

In [9]:
# y shape min
y_min = df_shape[1].min()
y_min

3338

In [10]:
# reshape the individual images using a using numpy slicing

reshape_image = []
for image in image_list:
    #print(image)
    reshape_image.append(image[:x_min,:y_min])

In [11]:
# verify the image shapes are equal 
for i in reshape_image:
    print(i.shape)

(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)
(3345, 3338, 3)


In [ ]:
# data type
type(reshape_image)

In [12]:
# create a variable for the with the dimensions for the resize
img_shp = reshape_image[0].shape
img_shp

(3345, 3338, 3)

In [13]:
# explore reshape image
len(reshape_image[0:])

18

In [14]:
# explore reshape image
len(reshape_image[0:][0])

3345

In [15]:
# explore reshape image
len(reshape_image[0:][0])

3345

In [16]:
img_shp

(3345, 3338, 3)

In [17]:
img_shp[0]

3345

In [18]:
img_shp[1]

3338

In [19]:
for v in reshape_image:
    print(v.max())

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255


##set up image resize
def image_resize(scale_percent):
    scale_percent = scale_percent
    


In [ ]:
# set up variables for the image resize

scale_percent = 50
width = int(img_shp[1] * scale_percent / 100)
height = int(img_shp[0] * scale_percent / 100)
dim = (width, height)

In [ ]:
reshape_image[0].shape

In [ ]:
# resize individual images in a folder
ls = []
for ind_i in reshape_image:
    ls.append(cv2.resize(ind_i, dim, interpolation = cv2.INTER_AREA))
    

In [ ]:
# check that the resize worked correctly
for i in ls:
    print(i.shape)

### Getting the target class Water

In [ ]:
#open geojson training folder
file = '/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/train_geojson_v3/'

for name in os.listdir(file):
    files = os.walk(file)
    geojson_files = [f[0] for f in os.walk(file)]

In [ ]:
#list of all the folders inside the training folder
geojson_folders = os.listdir(geojson_files[0])
geojson_folders

In [ ]:
geojson_files[0]

In [ ]:
# create a dictionary with key:image, and values:class
geojson_per_image = {}

for i in geojson_folders:
    fichiers = os.listdir(geojson_files[0]+ f"/{i}")
    geojson_per_image[i] = int(any('008' in s for s in fichiers)) #target water

In [ ]:
# print the dict to verify
geojson_per_image

In [ ]:
# put the dict into a pandas dataframe
df_target = pd.DataFrame.from_dict(geojson_per_image, orient='index')
df_target

In [ ]:
# explore the images and classes
len(df_target)

In [ ]:
y_train = df_target.head(23)
y_train

In [ ]:
len(ls)

In [ ]:
X_train = np.asanyarray(ls)
X_train.shape

## Build the Model

In [ ]:
ls[0].shape

In [ ]:
lg_n = []
for i in ls:
    lg_n.append(i.max())
    print(i.max())

In [ ]:
max(lg_n)

In [ ]:
X_train[0].shape

In [ ]:
# Home made model
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./130, input_shape=(X_train[0].shape)))

    model.add(layers.Conv2D(16, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=4, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

model = load_own_model()
model.summary()

In [ ]:
es = EarlyStopping(mode='max', patience=5, restore_best_weights=True)

model.fit(X_train, y_train,
         batch_size=16,validation_split=0.1, 
         epochs=25,verbose=1)

## First Run
Epoch 1/10
2/2 [==============================] - 148s 21s/step - loss: 1.2738 - accuracy: 0.6087
Epoch 2/10
2/2 [==============================] - 122s 17s/step - loss: 0.8995 - accuracy: 0.4783
Epoch 3/10
2/2 [==============================] - 130s 19s/step - loss: 1.0050 - accuracy: 0.3913
Epoch 4/10
2/2 [==============================] - 118s 21s/step - loss: 0.8076 - accuracy: 0.3913
Epoch 5/10
2/2 [==============================] - 145s 16s/step - loss: 0.6834 - accuracy: 0.3478
Epoch 6/10
2/2 [==============================] - 141s 20s/step - loss: 0.6296 - accuracy: 0.6087
Epoch 7/10
2/2 [==============================] - 120s 14s/step - loss: 0.6499 - accuracy: 0.6087
Epoch 8/10
2/2 [==============================] - 137s 17s/step - loss: 0.6765 - accuracy: 0.6087
Epoch 9/10
2/2 [==============================] - 114s 19s/step - loss: 0.6822 - accuracy: 0.6087
Epoch 10/10
2/2 [==============================] - 125s 15s/step - loss: 0.6511 - accuracy: 0.6087
<tensorflow.python.keras.callbacks.History at 0x162800cd0>

In [ ]:
y_train.value_counts().max()

In [ ]:
baseline_model = y_train.value_counts().max()/len(y_train)
baseline_model

In [3]:
joblib.dump(model, 'model.joblib')

NameError: name 'model' is not defined